# comercio_ext_indices.tb_referencia_ncm_ppi
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPI_delta`
## 📌 Descrição do arquivo

Classificação **PPI** (PT/EN) para traduzir o campo `CO_PPI` da dimensão NCM.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_PPI`|Código PPI|Identificador da categoria PPI|
|`NO_PPI`|Nome (PT)|Descrição do PPI em português|
|`NO_PPI_MIN`|Nome (PT – minúsculo)|Variante textual|
|`NO_PPI_ING`|Nome (EN)|Descrição em inglês|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/comercio_ext_indices/NCM_PPI_delta/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_ppi/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_ppi"



In [0]:
silverSchema = T.StructType([
    T.StructField("CO_PPI",     T.StringType(),   False),
    T.StructField("NO_PPI",     T.StringType(),   False),
    T.StructField("NO_PPI_MIN", T.StringType(),   True ),
    T.StructField("NO_PPI_ING", T.StringType(),   True ),
    T.StructField("TS_REF",     T.TimestampType(),False),
    T.StructField("NM_ORIGEM",  T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_PPI", up("CO_PPI"))
    .withColumn("NO_PPI", F.col("NO_PPI").cast(T.StringType()))
    .withColumn("NO_PPI_MIN", F.col("NO_PPI_MIN").cast(T.StringType()))
    .withColumn("NO_PPI_ING", F.col("NO_PPI_ING").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/ncm_ppi"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_PPI").isNotNull())


In [0]:

df_dedup = df_valid.dropDuplicates(["CO_PPI"])

In [0]:
df_silver = df_dedup.select("CO_PPI","NO_PPI","NO_PPI_MIN","NO_PPI_ING","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_estatisticas.tb_exportacoes")

merge_condition = """
  t.CO_PPI   = s.CO_PPI
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_PPI": "s.NO_PPI",
        "NO_PPI_MIN": "s.NO_PPI_MIN",
        "NO_PPI_ING": "s.NO_PPI_ING",
        "TS_REF":     "s.TS_REF",
        "NM_ORIGEM":  "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)